In [1]:
import os 
import sys
import glob
import numpy as np
import itk
from itk import TubeTK as ttk
from itkwidgets import view

In [2]:
# NRRD Study Name
studyname = 'G:/My Drive/Projects/Proj_UNC_StrokeCollaterals/Experiments/UNC/CTP/CTAT-003'

# NRRD Files
directory = (studyname + '/')

# Saved NRRD Files 
directory2 = (studyname + '-Reg/')

# Mask Creation and Location
directory3 = (studyname + '-MinMax/')

pic_folder = os.listdir(directory)
pic_folder = [pic_folder for pic_folder in pic_folder if ".nii" in pic_folder]
pic_folder.sort()
print(pic_folder)
num_images = len(pic_folder)

im0 = itk.imread(directory + pic_folder[int(num_images/2)], itk.F)
immath = ttk.ImageMath.New(Input=im0)
immath.Blur(0.5)
im0Blur = immath.GetOutput()

immath.Threshold(150, 800, 1, 0)
immath.Dilate(10, 1, 0)
mask0 = immath.GetOutputUChar()
mask0Tmp = itk.GetArrayFromImage(mask0)
mask0Tmp[0:4,:,:] = 0
sizeZ = mask0Tmp.shape[0]
mask0Tmp[sizeZ-4:sizeZ,:,:] = 0
mask0 = itk.GetImageFromArray(mask0Tmp)
mask0.CopyInformation(im0)

itk.imwrite(mask0, directory3 + 'mask.mha', compression=True)
maskObj = itk.ImageMaskSpatialObject[3].New()
maskObj.SetImage(mask0)
maskObj.Update()

['CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_10.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_11.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_12.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_13.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_14.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_15.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_16.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_17.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_18.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_19.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_20.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_21.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_22.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_23.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_24.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_25.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_26.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_27.nii', 'CTAT-003-Dawn_DAWN_PERFUSI

In [3]:
view(mask0)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [ ]:
Dimension = 3
PixelType = itk.ctype('float')
ImageType = itk.Image[PixelType, Dimension]

imdatamax = itk.GetArrayFromImage(im0)
imdatamin = imdatamax

imMoving0Blur = im0Blur

for imNum in range(num_images):
    imMoving = itk.imread( directory + pic_folder[imNum], itk.F )
    
    immath.SetInput(imMoving)
    immath.Blur(0.5)
    imMovingBlur = immath.GetOutput()
    
    imreg = ttk.RegisterImages[ImageType].New()
    imreg.SetFixedImage(imMoving0Blur)
    imreg.SetMovingImage(imMovingBlur)
    imreg.SetRigidMaxIterations(3000)
    imreg.SetRegistration("RIGID")
    imreg.SetExpectedOffsetMagnitude(20)
    imreg.SetExpectedRotationMagnitude(0.3)
    imreg.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    imreg.SetFixedImageMaskObject(maskObj)
    imreg.SetSampleFromOverlap(True)
    imreg.SetReportProgress(True)
    imreg.Update()
    tfm = imreg.GetCurrentMatrixTransform()
    imMoving0Blur = imreg.GetFinalMovingImage("LINEAR_INTERPOLATION", -1024)
    imMoving0 = imreg.ResampleImage("LINEAR_INTERPOLATION", imMoving, tfm, -1024)
    
    itk.imwrite( imMoving0, directory2 + pic_folder[imNum], compression=True )
    

    print(tfm)
    
    imdataTmp = itk.GetArrayFromImage(imMoving0)
    imdatamax = np.maximum(imdatamax,imdataTmp)
    imdatamin = np.minimum(imdatamin,imdataTmp)
    
    #out = itk.GetImageFromArray(imdatamax)
    #out.CopyInformation(im0)
    #itk.imwrite(out, (directory3 + 'max' + str(imNum) + '.nrrd'))
    
    percent = (imNum + 1) / num_images * 100
    print(str(round(percent)) + '% : ' + pic_folder[imNum])
    
print('Done')    

AffineTransform (000001FABEFF0BE0)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 1459813
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999998 0.00170899 -0.000626251 
    -0.00171006 0.999997 -0.00171297 
    0.000623322 0.00171404 0.999998 
  Offset: [0.224218, -0.0611026, -0.155309]
  Center: [0.21582, -164.716, 3.50005]
  Translation: [-0.0594717, -0.0669847, -0.437509]
  Inverse: 
    0.999998 -0.00171006 0.000623322 
    0.00170899 0.999997 0.00171404 
    -0.000626251 -0.00171297 0.999998 
  Singular: 0

4% : CTAT-001-Perf_DAWN_PERFUSION_20180301063224_10.nii
AffineTransform (000001FABED63FD0)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 2923970
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999997 0.00235968 -0.000292841 
    -0.00235989 0.999997 -0.000734238 
    0.000291107 0.000734927 1 
  Offset: [0.278939, -0.0606089, -0.268699]
  Cen

In [5]:
out = itk.GetImageFromArray(imdatamax)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'max.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamin)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'min.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamax - imdatamin)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'diff.nrrd'), compression=True)

print('Done')

Done


In [6]:
view(out)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…